In [1]:
#import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

import re

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.model_selection import train_test_split

# Import Data:

In [2]:
data_1 = pd.read_csv("analytic_data2019.csv", skiprows=0)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,1,2,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,2

In [3]:
data_1 = data_1.iloc[2:]

In [4]:
data_1_FFIPS = list(data_1['5-digit FIPS Code'])
# data_1_FFIPS

In [5]:
# cols1 = data_1.columns
# for col in cols1:
#     print(col)

In [6]:
xls = pd.ExcelFile('2019 County Health Rankings Data - v2.xls')
# sheet0 = xls.parse(0,skiprows=1) 
# sheet1 = xls.parse(1,skiprows=1) 
# sheet2 = xls.parse(2,skiprows=1) 
sheet3 = xls.parse(3,skiprows=1) 
# sheet4 = xls.parse(4,skiprows=1) 
sheet5 = xls.parse(5,skiprows=1) 
# sheet6 = xls.parse(6,skiprows=1) 

## Merge Datasets into single DataFrames

There are stats for the whole state with in the `data_1` set. Therefore we need to remove the rows which in `Name` does not contain county.

In [12]:
states = ['Alabama', 'Alaska', 'Arizona' ,'Arkansas', 'California',
          'Colorado','Connecticut','Delaware','Florida','Georgia',
          'Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky',
          'Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota',
          'Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire',
          'New Jersey','New Mexico','New York','North Carolina','North Dakota',
          'Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina',
          'South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington',
          'West Virginia','Wisconsin','Wyoming']

In [8]:
print(data_1.shape, sheet3.shape, sheet5.shape)

(3193, 534) (3142, 162) (3142, 111)


The miss match between the CSV and the Excel rows number, is the states averages and the US average that need to be removed from `data_1`

- Set `Name` as the index
- Transpose the DF
- Remove US & States using columns remove
- Transpose DF back
- Rest index

In [9]:
data_1.set_index('Name', drop=True, inplace =True)

In [10]:
data_1 = data_1.T

In [17]:
cols = list(data_1.columns)
cols_new = [x for x in cols if x not in states]
data_1 = data_1[cols_new]
data_1.head()

Name,Autauga County,Baldwin County,Baldwin County,Barbour County,Barbour County,Bibb County,Bibb County,Blount County,Blount County,Bullock County,...,Sheridan County,Sheridan County,Sublette County,Sweetwater County,Teton County,Teton County,Teton County,Uinta County,Washakie County,Weston County
State FIPS Code,01,01,13,01,54,01,13,01,47,01,...,38,56,56,56,16,30,56,56,56,56
County FIPS Code,001,003,009,005,1,007,021,009,9,011,...,83,33,35,37,081,99,39,41,43,45
5-digit FIPS Code,01001,01003,13009,01005,54001,01007,13021,01009,47009,01011,...,38083,56033,56035,56037,16081,30099,56039,56041,56043,56045
State Abbreviation,AL,AL,GA,AL,WV,AL,GA,AL,TN,AL,...,ND,WY,WY,WY,ID,MT,WY,WY,WY,WY
Release Year,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,...,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019


In [19]:
# data_1 = data_1.T
data_1.reset_index(inplace=True)
data_1.head()

,Name,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,...,Male population 18-44 raw value,Male population 45-64 raw value,Male population 65+ raw value,Total male population raw value,Female population 0-17 raw value,Female population 18-44 raw value,Female population 45-64 raw value,Female population 65+ raw value,Total female population raw value,Population growth raw value
0,Autauga County,01,001,01001,AL,2019,1,8824.0571232,815,156132,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Baldwin County,01,003,01003,AL,2019,1,7224.6321603,2827,576496,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Baldwin County,13,009,13009,GA,2019,1,9659.2241558,711,127507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Barbour County,01,005,01005,AL,2019,1,9586.165037,451,72222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barbour County,54,1,54001,WV,2019,1,9319.74,281,46152,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
data_1.shape

(14529, 534)

In [21]:
len(cols_new)

3143

In [22]:
cols_x = set(list(cols_new))
len(cols_x)

1877

### Merge Data
- Set index as FIFPS code
- Concat on axis=0

In [9]:
data_1.set_index('5-digit FIPS Code', inplace=True)
sheet3.set_index('FIPS', inplace=True)
sheet5.set_index('FIPS', inplace=True)

In [8]:
data_1_Names = list(data_1.Name)

In [16]:
combimed = pd.concat([data_1, sheet5, sheet3], axis=1, sort=False, join='inner')
combimed.shape

(2138, 804)

In [17]:
combimed.head()

,State FIPS Code,County FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,Premature death CI low,...,95% CI - High.17,Quartile.33,% Drive Alone (Black),% Drive Alone (Hispanic),% Drive Alone (White),# Workers who Drive Alone,% Long Commute - Drives Alone,95% CI - Low.18,95% CI - High.18,Quartile.34
21023,21,23,KY,Bracken County,2019,1,14928.2,170,23400,11738.3,...,86.071579,2,NaN,NaN,NaN,2831,57.1,48.576061,65.623939,4
21025,21,25,KY,Breathitt County,2019,1,15397.5,362,37327,13037.6,...,84.148410,1,NaN,NaN,NaN,3034,36.8,30.425810,43.174190,3
21027,21,27,KY,Breckinridge County,2019,1,10021.5,404,55848,8413.79,...,78.655553,1,NaN,NaN,NaN,5660,41.4,36.098207,46.701793,3
21029,21,29,KY,Bullitt County,2019,1,7934.86,1091,225641,7202.84,...,86.865017,4,88.159311,70.746269,79.418516,32148,47.7,44.570213,50.829787,4
21031,21,31,KY,Butler County,2019,1,11348.7,249,35726,9188.25,...,87.569632,3,NaN,NaN,NaN,4210,41.3,35.325896,47.274104,3


In [18]:
cols1 = combimed.columns
cols1 = sorted(cols1)
for col in cols1:
    print(col)

# African American
# Alcohol-Impaired Driving Deaths
# American Indian/Alaskan Native
# Asian
# Associations
# Chlamydia Cases
# Deaths
# Deaths.1
# Deaths.2
# Dentists
# Driving Deaths
# Drug Overdose Deaths
# Firearm Fatalities
# Food Insecure
# HIV Cases
# Hispanic
# Homeowners
# Households
# Households with Severe Cost Burden
# Injury Deaths
# Limited Access
# Mental Health Providers
# Motor Vehicle Deaths
# Native Hawaiian/Other Pacific Islander
# Non-Hispanic White
# Not Proficient in English
# Primary Care Physicians
# Rural
# Single-Parent Households
# Some College
# Unemployed
# Uninsured
# Uninsured
# Uninsured.1
# Workers who Drive Alone
% 65 and older CI high
% 65 and older CI low
% 65 and older denominator
% 65 and older numerator
% 65 and older raw value
% 65 and over
% < 18
% African American
% Alcohol-Impaired
% American Indian and Alaskan Native CI high
% American Indian and Alaskan Native CI low
% American Indian and Alaskan Native denominator
% American Indian and Al

In [19]:
# Manipulate column names
subs = [(' ', '_'),('.',''),("'",""),('™', ''), ('®',''),
        ('+','plus'), ('½','half'), ('-','_'), ('#','Num_'), 
        ('%', 'Per_'), ('<', 'Greater_than_'), ('>', 'Less_than_')
       ]

def col_formatting(col):
    new_cols = []
    for line in col:
        for sub in subs:
            line = line.replace(sub[0], sub[1])
        new_cols.append(line)
    return new_cols

In [20]:
col_new = col_formatting(combimed.columns)
combimed.columns = col_new

In [21]:
cols1 = combimed.columns
cols1 = sorted(cols1)
for col in cols1:
    print(col)

20th_Percentile_Income
80th_Percentile_Income
95Per__CI___High
95Per__CI___High
95Per__CI___High1
95Per__CI___High1
95Per__CI___High10
95Per__CI___High10
95Per__CI___High11
95Per__CI___High11
95Per__CI___High12
95Per__CI___High12
95Per__CI___High13
95Per__CI___High13
95Per__CI___High14
95Per__CI___High14
95Per__CI___High15
95Per__CI___High15
95Per__CI___High16
95Per__CI___High16
95Per__CI___High17
95Per__CI___High18
95Per__CI___High2
95Per__CI___High2
95Per__CI___High3
95Per__CI___High3
95Per__CI___High4
95Per__CI___High4
95Per__CI___High5
95Per__CI___High5
95Per__CI___High6
95Per__CI___High6
95Per__CI___High7
95Per__CI___High7
95Per__CI___High8
95Per__CI___High8
95Per__CI___High9
95Per__CI___High9
95Per__CI___Low
95Per__CI___Low
95Per__CI___Low1
95Per__CI___Low1
95Per__CI___Low10
95Per__CI___Low10
95Per__CI___Low11
95Per__CI___Low11
95Per__CI___Low12
95Per__CI___Low12
95Per__CI___Low13
95Per__CI___Low13
95Per__CI___Low14
95Per__CI___Low14
95Per__CI___Low15
95Per__CI___Low15
95Per__CI_

## Remove columns that are % , or confidence intervals

- should you just have raw values?

In [ ]:
clean_columns = [x for x in df.columns if
                 (“numerator” not in x) and
                 (“denominator” not in x) and
                 (“CI_low” not in x) and
                 (“CI_high” not in x) and
                 (“_(White)” not in x) and
                 (“_(Black)” not in x) and
                 (“_(Hispanic)” not in x) and 
                 (“CI” not in x) and 
                 (“Rate” not in x) and 
                 (“Ratio” not in x)]

## Data Exploration:

Create dataframe for description of columns + Re import , drop first row 

In [11]:
def data_nUnique_values(df):
    print("DataFrame Shape:", df.shape)
    print("----------------------------")
    for n in df.columns:
        if df[n].isna().sum() != df.shape[0]:
            print(n, df[n].nunique(), df[n].isna().sum())

In [12]:
# data_nUnique_values(combimed)

## Set Target and Predictors:

In [13]:
outcomes = ['Life_Expectancy',
 'Life_Expectancy_(Black)',
 'Life_Expectancy_(Hispanic)',
 'Life_Expectancy_(White)',
 'Life_expectancy_(Black)',
 'Life_expectancy_(Hispanic)',
 'Life_expectancy_(White)',
 'Life_expectancy_CI_high',
 'Life_expectancy_CI_low',
 'Life_expectancy_denominator',
 'Life_expectancy_numerator',
 'Life_expectancy_raw_value']

predictors = list(combimed.columns)

In [14]:
predictors = [x for x in predictors if x not in outcomes]

In [15]:
df_y = combimed[['Life_Expectancy',
 'Life_Expectancy_(Black)',
 'Life_Expectancy_(Hispanic)',
 'Life_Expectancy_(White)',
 'Life_expectancy_(Black)',
 'Life_expectancy_(Hispanic)',
 'Life_expectancy_(White)',
 'Life_expectancy_CI_high',
 'Life_expectancy_CI_low',
 'Life_expectancy_denominator',
 'Life_expectancy_numerator',
 'Life_expectancy_raw_value']]

In [16]:
df_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3195 entries, 0 to 3194
Data columns (total 12 columns):
Life_Expectancy                3073 non-null float64
Life_Expectancy_(Black)        1339 non-null float64
Life_Expectancy_(Hispanic)     1247 non-null float64
Life_Expectancy_(White)        1726 non-null float64
Life_expectancy_(Black)        1340 non-null object
Life_expectancy_(Hispanic)     1248 non-null object
Life_expectancy_(White)        1727 non-null object
Life_expectancy_CI_high        3124 non-null object
Life_expectancy_CI_low         3124 non-null object
Life_expectancy_denominator    0 non-null object
Life_expectancy_numerator      0 non-null object
Life_expectancy_raw_value      3124 non-null object
dtypes: float64(4), object(8)
memory usage: 299.7+ KB


In [17]:
def data_type(df, cols, datatype='str'):
    for col in cols:
        df[col] = df[col].astype(datatype)
    
    return df.info()

In [18]:
data_type(df_y, outcomes, 'float')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3195 entries, 0 to 3194
Data columns (total 12 columns):
Life_Expectancy                3073 non-null float64
Life_Expectancy_(Black)        1339 non-null float64
Life_Expectancy_(Hispanic)     1247 non-null float64
Life_Expectancy_(White)        1726 non-null float64
Life_expectancy_(Black)        1340 non-null float64
Life_expectancy_(Hispanic)     1248 non-null float64
Life_expectancy_(White)        1727 non-null float64
Life_expectancy_CI_high        3124 non-null float64
Life_expectancy_CI_low         3124 non-null float64
Life_expectancy_denominator    0 non-null float64
Life_expectancy_numerator      0 non-null float64
Life_expectancy_raw_value      3124 non-null float64
dtypes: float64(12)
memory usage: 299.7 KB


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
life_mean = df_y.mean(axis=1)

Target = life_mean

predictors = "everything else"

# Create Model DataFrame

In [22]:
df_model = combimed[predictors]

In [23]:
df_model['Life_Expectancy_Mean'] = life_mean

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
df_model.head()

,State_FIPS_Code,County_FIPS_Code,5_digit_FIPS_Code,State_Abbreviation,Name,Release_Year,County_Ranked_(Yes=1/No=0),Premature_death_raw_value,Premature_death_numerator,Premature_death_denominator,...,Quartile33,Per__Drive_Alone_(Black),Per__Drive_Alone_(Hispanic),Per__Drive_Alone_(White),Num__Workers_who_Drive_Alone,Per__Long_Commute___Drives_Alone,95Per__CI___Low18,95Per__CI___High18,Quartile34,Life_Expectancy_Mean
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,82.722037,NaN,83.710021,20911.0,38.3,34.362673,42.237327,3,75.756145
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,84.921904,68.343876,83.799135,74415.0,40.5,38.183480,42.816520,3,82.701328
2,01,000,01000,AL,Alabama,2019,NaN,9917.2328984,80440,13636816,...,1,80.154701,NaN,86.245265,7242.0,33.8,28.867176,38.732824,2,75.637862
3,01,001,01001,AL,Autauga County,2019,1,8824.0571232,815,156132,...,3,NaN,NaN,NaN,6930.0,48.6,40.310796,56.889204,4,75.032760
4,01,003,01003,AL,Baldwin County,2019,1,7224.6321603,2827,576496,...,3,NaN,81.767181,83.414506,18426.0,59.7,55.480714,63.919286,4,79.475268


# Train Test Split

In [ ]:
train, test = train_test_split(df_model, test_size=34)

In [ ]:
# Your code here - Fit the actual model
print("Train:", len(train),"\nTest:", len(test), "\n%_test:",(len(test)/(len(test)+len(train)))*100 )